In [ ]:
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, find_peaks
import numpy as np
import datetime
import pandas as pd
import obspy
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import time
import glob
import os
import pytz
from tqdm import tqdm
import csv
import matplotlib.dates as mdates
import csv
import re
from scipy.stats import norm
from templatematching import *
from getanalisisfiles import *




In [ ]:
#testing fuctions
base_path = "/data/fast1/veronica-scratch-rainier-downsampling/drive1_ds"
start_date = "2023-08-25_00"
end_date = "2023-08-25_00"

# using the fuction get_file_list 
file_list = get_file_list(base_path,start_date,end_date)

len(file_list)
file_list[-1]

In [ ]:
#path of the templates

template_list = glob.glob ('/data/data4/veronica-scratch-rainier/swarm_august2023/test-template-maker/test_4sec_templates/filtering/*')


In [ ]:
# Base directory to save files CC
base_directory_cc = '/data/data4/veronica-scratch-rainier/swarm_august2023/results_CC_TMA/'

# Variable folder name
template_size = 21  #20 #21 frankesitein, duration of the template (6 for test)
folder_name = f'CC_{template_size}sec-templates'

# Full path
full_path = os.path.join(base_directory_cc, folder_name)

# Create folder if it doesn't exist
if not os.path.exists(full_path):
    os.makedirs(full_path)
print(full_path)

In [ ]:
# Parameters for the filter

chan_min = 0 # from channel 0
chan_max = 2500  #to channel 3000
channel_number = chan_max -chan_min
low_cut1 = 2
hi_cut1 = 9.0
#fs=attrs['MaximumFrequency']*2
fs = 20
samples_per_file = 60*fs
b, a = butter(2, (low_cut1, hi_cut1), 'bp', fs=fs)

# Buiding outputfiles and correlations for each template on the list

In [ ]:
process_files(file_list, template_list, chan_min, chan_max, channel_number, samples_per_file, b, a, full_path)

# ANALYSIS OF DATA

In [ ]:
#calculting the fuction
output_dir = '/data/data4/veronica-scratch-rainier/swarm_august2023/results_CC_TMA/h5_files_timestamps'
output_file_h5 = create_timestamps_h5(file_list, output_dir)

In [ ]:
## timestamps to datetimeobject

# opening the file that containg the timestamps
with h5py.File(output_file_h5, 'r') as f:
    timestamps_pt = np.array(f['timestamps'])

# Define the Pacific Timezone
pt_timezone = pytz.timezone('America/Los_Angeles')

# Convert timestamps from microseconds to seconds
timestamps_seconds = timestamps_pt / 1e6

# Convert timestamps to datetime objects in Pacific Time
datetime_objects_pt = [datetime.datetime.fromtimestamp(ts, pt_timezone) for ts in timestamps_seconds]

# Define the UTC Timezone
utc_timezone = pytz.timezone('UTC')

# Convert datetime objects to UTC
datetime_objects_utc = [dt_pt.astimezone(utc_timezone) for dt_pt in datetime_objects_pt]

# Redifining times variables
time_range = datetime_objects_utc

In [ ]:
# full_path is going to give you the CC data set for this run, also calcultin MAD so we can define the thresold

# Obtener una lista de todas las carpetas en el directorio base
folders = [folder for folder in os.listdir(full_path) if os.path.isdir(os.path.join(full_path, folder))]

# Inicializar una lista para almacenar los datos concatenados por cada carpeta
concatenated_data_per_folder = []

# Iterar sobre las carpetas y cargar los archivos .npy
for folder in folders:
    folder_path = os.path.join(full_path, folder)
    npy_files = [np.load(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.npy')]
    concatenated_data_per_folder.append(np.concatenate(npy_files, axis=0))

# calcuting mad for each folder and for defining the thresold later
mads_per_folder = {}
for folder, folder_data in zip(folders, concatenated_data_per_folder):
    det_shelly,mad = detections(folder_data,mode='shelly')
    thresh = np.round(np.abs(norm.ppf(1/fs/(60*60))))
    #print(f"MAD for folder {folder}: {mad}")
    print(f"thresold for folder {folder}: {thresh}")
    det_ind = np.where(np.abs(det_shelly) > thresh)[0]
    
#average_mad = np.mean(list(mads_per_folder.values()))


#print(f"average mad {average_mad}")

In [ ]:
# Directorio base donde se encuentran las carpetas
base_directory = '/data/data4/veronica-scratch-rainier/test_corr'

# Obtener una lista de todas las carpetas en el directorio base
folders = [folder for folder in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, folder))]

# Inicializar una lista para almacenar los datos concatenados por cada carpeta
concatenated_data_per_folder = []

# Iterar sobre las carpetas y cargar los archivos .npy
for folder in folders:
    folder_path = os.path.join(base_directory, folder)
    npy_files = [np.load(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.npy')]
    concatenated_data_per_folder.append(np.concatenate(npy_files, axis=0))

# Crear el rango de fechas o tiempo con exactamente 20 objetos datetime por segundo
# Asumiendo que ya tienes definido time_range adecuadamente

# Graficar los datos para cada carpeta
fig, axs = plt.subplots(len(folders), 1, figsize=(10, 5*len(folders)))

for i, folder_data in enumerate(concatenated_data_per_folder):
    axs[i].plot(time_range, folder_data, label=f'Folder {folders[i]}', color='blue', linestyle='-')
    axs[i].set_title(f'Template {folders[i]}')
    axs[i].set_xlabel('Time')
    axs[i].set_ylabel('Correlation Value')
    axs[i].legend()  # Add legend to each subplot
    axs[i].set_ylim([0,0.5])  # Set y-axis limits from 0 to 0.5

plt.tight_layout()
plt.grid()
plt.show()

# add thresold and detections stuff

In [ ]:

# Define los umbrales
thresholds = [0.5, 0.1, 0.05, 0.04, 0.035, 0.03]

# Directorio donde se guardarán los archivos de texto
output_directory = "textfiles_results_thresholds"

# Itera sobre cada carpeta
for folder, folder_data in zip(folders, concatenated_data_per_folder):
    # Crear directorio si no existe
    folder_output_directory = os.path.join(output_directory, folder)
    if not os.path.exists(folder_output_directory):
        os.makedirs(folder_output_directory)
    
    # Ruta del archivo de salida
    output_file_path = os.path.join(folder_output_directory, f"detections_results_{folder}.txt")
    
    # Abre el archivo de salida en modo escritura
    with open(output_file_path, 'w') as file:
        # Escribe los encabezados
        file.write("Threshold\tNumber of Detections\tDetection Times (UTC)\n")

        # Itera sobre cada umbral
        for threshold in thresholds:
            # Encuentra los índices donde los valores superan el umbral
            indices_above_threshold = np.where(np.abs(folder_data) > threshold)[0]
            diff_indices = np.diff(indices_above_threshold)
            group_changes = np.where(diff_indices > 10)[0]
            detection_groups = np.split(indices_above_threshold, group_changes + 1)

            # Itera sobre cada grupo de detección
            for group in detection_groups:
                # Verifica si el grupo no está vacío
                if len(group) > 0:
                    # Toma solo la primera fecha en cada grupo
                    first_detection_time_utc = time_range[group[0]].strftime('%Y-%m-%d_%H.%M.%S')
                    #first_detection_time_utc = datetime.datetime.utcfromtimestamp(time_range[group[0]].timestamp()).strftime('%Y-%m-%d_%H.%M.%S')
                    
                    # Escribe los datos en una fila separados por tabulaciones
                    file.write(f"{threshold}\t{len(detection_groups)}\t{first_detection_time_utc}\n")
                else:
                    print("Grupo vacío encontrado para el umbral:", threshold)

# Mensaje de confirmación
print(f"Los resultados se han guardado en {output_directory}")
